<a href="https://colab.research.google.com/github/sooonsyk/Pocari/blob/main/classification/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **base model**

In [ ]:
import os                                               #파일 접근
import torch                                            #신경망 사용
import torch.nn as nn
import numpy as np
import torch.optim as optim                             #최적화 알고리즘 제공
from torchvision import datasets, transforms            #데이터셋 사용, 이미지 데이터 전처리
from torch.utils.data import DataLoader                 #데이터 미니배치로 나누어서 로드
from sklearn.model_selection import train_test_split    #학습, 검증, 테스트 데이터 분리

# 이미지 전처리 및 데이터 증강
transform = transforms.Compose([
    transforms.Resize((128, 128)),          # 이미지 크기 조정
    transforms.ToTensor(),                  # 이미지를 텐서로 변환
    transforms.Normalize((0.5,), (0.5,))    # 정규화
])

# 이미지 데이터셋 로드
dataset = datasets.ImageFolder(root='/content/drive/MyDrive/images_0601', transform=transform)

# 데이터셋을 학습, 검증, 테스트로 분할
train_val_indices, test_indices = train_test_split(list(range(len(dataset))), test_size=0.2, random_state=42)
train_indices, val_indices = train_test_split(train_val_indices, test_size=0.25, random_state=42)  # train 60%, val 20%, test 20%

train_dataset = torch.utils.data.Subset(dataset, train_indices)
val_dataset = torch.utils.data.Subset(dataset, val_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)

# DataLoader 생성 - data 수 적어서 batch 지정 안 함
train_loader = DataLoader(dataset=train_dataset, shuffle=True)   # 오버피팅 방지, 일반화
val_loader = DataLoader(dataset=val_dataset, shuffle=False)      # 일관성
test_loader = DataLoader(dataset=test_dataset, shuffle=False)

# 데이터셋 클래스 확인
print("Classes:", dataset.classes)
print("Number of training samples:", len(train_dataset))
print("Number of validation samples:", len(val_dataset))
print("Number of test samples:", len(test_dataset))

Classes: ['blood', 'diarrhea', 'green', 'normal', 'white']
Number of training samples: 27
Number of validation samples: 10
Number of test samples: 10


In [ ]:
# 간단한 CNN 모델 정의
class SimpleCNN(nn.Module):

    # 모델 구조 정의
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, len(dataset.classes))

    # 데이터가 모델을 통과하는 과정 정의
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 32 * 32)    # 3D 텐서 1D텐서로 변환
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)                 # 최종 결과 출력
        return x

# 모델 초기화 및 손실 함수, 최적화 도구 설정
model = SimpleCNN()

criterion = nn.CrossEntropyLoss()                     # 다중 분류에 적합
optimizer = optim.Adam(model.parameters(), lr=0.001)  # 확률적 경사 하강법의 변형

# 학습 및 검증 함수 정의
def train(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()                         # 훈련모드
        running_loss = 0.0
        for inputs, labels in train_loader:   # 미니배치를 가져와 입력과 레이블 받음
            optimizer.zero_grad()             # 이전에 계산한 그래디언트 초기화
            outputs = model(inputs)           # 결과 출력
            loss = criterion(outputs, labels) # 손실 계산
            loss.backward()                   # 역전파로 그래디언트 업데이트
            optimizer.step()                  # 파라미터 업데이트
            running_loss += loss.item()

        # 학습 중 손실 출력
        print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {running_loss / len(train_loader)}')

        # 검증 데이터셋을 사용하여 모델 성능 평가
        model.eval()                          # 평가모드
        correct = 0
        total = 0
        with torch.no_grad():                 # 그래디언트 계산 진행하지 않음
            for inputs, labels in val_loader:
                outputs = model(inputs)                  # 결과 출력
                _, predicted = torch.max(outputs, 1)     # 각 데이터에 대해 예측된 레이블
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        val_accuracy = correct / total
        print(f'Validation Accuracy: {val_accuracy:.2%}')

# 모델 학습
train(model, criterion, optimizer, train_loader, val_loader)


Epoch [1/10], Train Loss: 1.7217538458881554
Validation Accuracy: 60.00%
Epoch [2/10], Train Loss: 0.7891250360343192
Validation Accuracy: 40.00%
Epoch [3/10], Train Loss: 0.301701779782619
Validation Accuracy: 60.00%
Epoch [4/10], Train Loss: 0.0685588334415612
Validation Accuracy: 40.00%
Epoch [5/10], Train Loss: 0.020239673532894433
Validation Accuracy: 40.00%
Epoch [6/10], Train Loss: 0.0009237195046580909
Validation Accuracy: 40.00%
Epoch [7/10], Train Loss: 0.0007025767062916404
Validation Accuracy: 40.00%
Epoch [8/10], Train Loss: 0.0005640616002269612
Validation Accuracy: 40.00%
Epoch [9/10], Train Loss: 0.00043823193444975713
Validation Accuracy: 40.00%
Epoch [10/10], Train Loss: 0.0003563228317983868
Validation Accuracy: 40.00%


In [ ]:
# 테스트 데이터셋을 사용하여 모델 평가
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    predicted_labels = []
    true_labels = []

    #블록 내에서 테스트 데이터로더를 반복하면서 각 미니배치에 대한 예측을 수행
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            # 예측된 레이블과 실제 레이블 저장
            predicted_labels.extend(predicted.tolist())
            true_labels.extend(labels.tolist())

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_accuracy = correct / total
    print(f'Test Accuracy: {test_accuracy:.2%}')
    return true_labels, predicted_labels

true_labels, predicted_labels = test(model, test_loader)

# 실제 레이블과 예측된 레이블 출력
print("True Labels:", true_labels)
print("Predicted Labels:", predicted_labels)

Test Accuracy: 70.00%
True Labels: [3, 4, 3, 4, 3, 4, 2, 3, 1, 3]
Predicted Labels: [0, 4, 3, 4, 3, 4, 2, 3, 2, 2]


## **white balance 전처리 추가**

In [ ]:
import os                                               #파일 접근
import torch                                            #신경망 사용
import torch.nn as nn
import numpy as np
import torch.optim as optim                             #최적화 알고리즘 제공
from torchvision import datasets, transforms            #데이터셋 사용, 이미지 데이터 전처리
from torch.utils.data import DataLoader                 #데이터 미니배치로 나누어서 로드
from sklearn.model_selection import train_test_split    #학습, 검증, 테스트 데이터 분리
from skimage import img_as_ubyte                        #이미지 처리 - white balance 에 활용
from PIL import Image

def white_patch(image, percentile=100):

    """
    Parameters
    ----------
    image : 입력으로 받는 이미지는 RGB 채널을 각각 가지는 (height, width, channels) 3차원 numpy 배열
    percentile : 채널 값의 보정값으로 사용하고자 하는 값을 지정하기 위한 비율, 기본값은 100으로 최대값을 사용
    """

    image = np.array(image)

    white_patch_image = img_as_ubyte(
        (image * 1.0 / np.percentile(image,
                                     percentile,
                                     axis=(0, 1))).clip(0, 1))
    return Image.fromarray(white_patch_image)


transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.Lambda(lambda x: white_patch(x, percentile=85)),  # 전처리 함수 적용
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # 정규화
])

# 이미지 데이터셋 로드
dataset = datasets.ImageFolder(root='/content/drive/MyDrive/images_0601', transform=transform)

# 데이터셋을 학습, 검증, 테스트로 분할
train_val_indices, test_indices = train_test_split(list(range(len(dataset))), test_size=0.2, random_state=42)
train_indices, val_indices = train_test_split(train_val_indices, test_size=0.25, random_state=42)  # train 60%, val 20%, test 20%

train_dataset = torch.utils.data.Subset(dataset, train_indices)
val_dataset = torch.utils.data.Subset(dataset, val_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)

# DataLoader 생성 - data 수 적어서 batch 지정 안 함
train_loader = DataLoader(dataset=train_dataset, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, shuffle=False)

# 데이터셋 클래스 확인
print("Classes:", dataset.classes)
print("Number of training samples:", len(train_dataset))
print("Number of validation samples:", len(val_dataset))
print("Number of test samples:", len(test_dataset))

Classes: ['blood', 'diarrhea', 'green', 'normal', 'white']
Number of training samples: 27
Number of validation samples: 10
Number of test samples: 10


In [ ]:
# 간단한 CNN 모델 정의
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 50 * 50, 128)  # 이 부분을 수정하여 두 번째 Conv 레이어의 출력 크기에 맞춰야 합니다.
        self.fc2 = nn.Linear(128, len(dataset.classes))

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 50 * 50)  # 두 번째 Conv 레이어를 통과한 후의 출력 크기에 맞추어 수정합니다.
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 모델 초기화 및 손실 함수, 최적화 도구 설정
model = SimpleCNN()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 및 검증 함수 정의
def train(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # 학습 중 손실 출력
        print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {running_loss / len(train_loader)}')

        # 검증 데이터셋을 사용하여 모델 성능 평가
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        val_accuracy = correct / total
        print(f'Validation Accuracy: {val_accuracy:.2%}')

# 모델 학습
train(model, criterion, optimizer, train_loader, val_loader)


Epoch [1/10], Train Loss: 2.167915595160728
Validation Accuracy: 60.00%
Epoch [2/10], Train Loss: 1.2100594691173345
Validation Accuracy: 20.00%
Epoch [3/10], Train Loss: 0.6055220785136852
Validation Accuracy: 50.00%
Epoch [4/10], Train Loss: 0.10382025867241922
Validation Accuracy: 50.00%
Epoch [5/10], Train Loss: 0.014618972726094836
Validation Accuracy: 50.00%
Epoch [6/10], Train Loss: 0.004240569718797781
Validation Accuracy: 50.00%
Epoch [7/10], Train Loss: 0.0014227716425239123
Validation Accuracy: 50.00%
Epoch [8/10], Train Loss: 0.0008523498988526828
Validation Accuracy: 50.00%
Epoch [9/10], Train Loss: 0.0005332201536650364
Validation Accuracy: 50.00%
Epoch [10/10], Train Loss: 0.0003883686386948347
Validation Accuracy: 50.00%


In [ ]:
# 테스트 데이터셋을 사용하여 모델 평가
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    predicted_labels = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            # 예측된 레이블과 실제 레이블 저장
            predicted_labels.extend(predicted.tolist())
            true_labels.extend(labels.tolist())

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_accuracy = correct / total
    print(f'Test Accuracy: {test_accuracy:.2%}')
    return true_labels, predicted_labels

true_labels, predicted_labels = test(model, test_loader)

# 실제 레이블과 예측된 레이블 출력
print("True Labels:", true_labels)
print("Predicted Labels:", predicted_labels)

Test Accuracy: 70.00%
True Labels: [3, 4, 3, 4, 3, 4, 2, 3, 1, 3]
Predicted Labels: [3, 4, 3, 4, 2, 4, 2, 3, 2, 2]


### **이미지 증강 추가**

In [ ]:
import os
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from skimage import img_as_ubyte
from PIL import Image

def white_patch(image, percentile=100):

    """
    Parameters
    ----------
    image : 입력으로 받는 이미지는 RGB 채널을 각각 가지는 (height, width, channels) 3차원 numpy 배열
    percentile : 채널 값의 보정값으로 사용하고자 하는 값을 지정하기 위한 비율, 기본값은 100으로 최대값을 사용
    """

    image = np.array(image)

    white_patch_image = img_as_ubyte(
        (image * 1.0 / np.percentile(image,
                                     percentile,
                                     axis=(0, 1))).clip(0, 1))
    return Image.fromarray(white_patch_image)

transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.Lambda(lambda x: white_patch(x, percentile=85)),  # 전처리 함수 적용
    transforms.RandomHorizontalFlip(),  # 랜덤으로 이미지를 수평으로 뒤집기
    transforms.RandomVerticalFlip(p=0.5), # 랜덤으로 이미지를 수직으로 뒤집기
    transforms.RandomRotation(degrees=(-10, 10)), # 랜덤으로 이미지 회전
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # 정규화
])

# 이미지 데이터셋 로드
dataset = datasets.ImageFolder(root='/content/drive/MyDrive/images_0601', transform=transform)

# 데이터셋을 학습, 검증, 테스트로 분할
train_val_indices, test_indices = train_test_split(list(range(len(dataset))), test_size=0.2, random_state=1)
train_indices, val_indices = train_test_split(train_val_indices, test_size=0.25, random_state=1)  # train 60%, val 20%, test 20%

train_dataset = torch.utils.data.Subset(dataset, train_indices)
val_dataset = torch.utils.data.Subset(dataset, val_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)

# DataLoader 생성 - data 수 적어서 batch 지정 안 함
train_loader = DataLoader(dataset=train_dataset, batch_size = 1, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size = 1, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size = 1, shuffle=False)

# 데이터셋 클래스 확인
print("Classes:", dataset.classes)
print("Number of training samples:", len(train_dataset))
print("Number of validation samples:", len(val_dataset))
print("Number of test samples:", len(test_dataset))

Classes: ['blood', 'diarrhea', 'green', 'normal', 'white']
Number of training samples: 27
Number of validation samples: 10
Number of test samples: 10


In [ ]:
# 간단한 CNN 모델 정의
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 50 * 50, 128)
        self.fc2 = nn.Linear(128, len(dataset.classes))

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 50 * 50)  # 두 번째 Conv 레이어를 통과한 후의 출력 크기에 맞추어 수정합니다.
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 모델 초기화 및 손실 함수, 최적화 도구 설정
model = SimpleCNN()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 및 검증 함수 정의
def train(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # 학습 중 손실 출력
        print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {running_loss / len(train_loader)}')

        # 검증 데이터셋을 사용하여 모델 성능 평가
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        val_accuracy = correct / total
        print(f'Validation Accuracy: {val_accuracy:.2%}')

# 모델 학습
train(model, criterion, optimizer, train_loader, val_loader)


Epoch [1/10], Train Loss: 2.676086973674871
Validation Accuracy: 80.00%
Epoch [2/10], Train Loss: 1.341817690818398
Validation Accuracy: 80.00%
Epoch [3/10], Train Loss: 1.148970368663194
Validation Accuracy: 90.00%
Epoch [4/10], Train Loss: 1.248877362796554
Validation Accuracy: 90.00%
Epoch [5/10], Train Loss: 1.048879081348854
Validation Accuracy: 80.00%
Epoch [6/10], Train Loss: 1.13217444485269
Validation Accuracy: 90.00%
Epoch [7/10], Train Loss: 0.8422969690366665
Validation Accuracy: 90.00%
Epoch [8/10], Train Loss: 0.6591349620939582
Validation Accuracy: 60.00%
Epoch [9/10], Train Loss: 0.6768748494954269
Validation Accuracy: 70.00%
Epoch [10/10], Train Loss: 0.7099919161603886
Validation Accuracy: 90.00%


In [ ]:
# 테스트 데이터셋을 사용하여 모델 평가
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    predicted_labels = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            # 예측된 레이블과 실제 레이블 저장
            predicted_labels.extend(predicted.tolist())
            true_labels.extend(labels.tolist())

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_accuracy = correct / total
    print(f'Test Accuracy: {test_accuracy:.2%}')
    return true_labels, predicted_labels

true_labels, predicted_labels = test(model, test_loader)

# 실제 레이블과 예측된 레이블 출력
print("True Labels:", true_labels)
print("Predicted Labels:", predicted_labels)

Test Accuracy: 40.00%
True Labels: [3, 3, 4, 4, 0, 1, 3, 3, 4, 3]
Predicted Labels: [4, 1, 4, 2, 3, 4, 3, 3, 2, 3]
